<a href="https://colab.research.google.com/github/hasitha-ranasinghe/tensorflow-colaboratory/blob/main/nlp_rnn_play_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [ ]:
# To select your own file
from google.colab import files
path_to_file = list(files.upload().keys())[0]

In [5]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [7]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [8]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [10]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [12]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [13]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [14]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [16]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [17]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 2.36994340e-04  1.55325083e-03 -7.75750866e-03 ...  6.22268417e-04
    1.05456272e-02  5.27220080e-04]
  [ 7.36984564e-03  1.49009028e-03 -2.55493377e-03 ...  4.35155496e-04
    3.29645304e-03 -1.09194801e-03]
  [ 1.19855227e-02 -2.87497882e-03  1.85470027e-03 ... -6.77585881e-03
    2.24516168e-03  5.65213570e-03]
  ...
  [ 3.18386033e-03 -6.27589505e-03 -1.81069027e-03 ... -9.11282003e-03
   -4.03339509e-03 -3.37915902e-04]
  [-1.41813257e-03 -2.44113128e-03 -2.02966708e-04 ... -8.59972183e-03
   -3.41655314e-03 -4.80640447e-03]
  [-4.54862369e-03 -5.53523190e-03 -4.63758223e-03 ... -9.88623872e-03
   -2.79618660e-03 -4.09486331e-03]]

 [[ 6.95683574e-03 -3.03754350e-03 -7.59510184e-03 ... -2.49892729e-03
    3.26617085e-03 -2.76309147e-04]
  [ 6.11911761e-03 -6.01018197e-04 -1.31417327e-02 ... -1.08024711e-03
    1.14360424e-02  8.84907378e-04]
  [-8.52478319e-04  1.63406949e-03 -1.08720614e-02 ... -4.69853636e-03
    6.18849602e-03 -3.24552436e-03]
  ...
  [ 1.281

In [18]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00023699  0.00155325 -0.00775751 ...  0.00062227  0.01054563
   0.00052722]
 [ 0.00736985  0.00149009 -0.00255493 ...  0.00043516  0.00329645
  -0.00109195]
 [ 0.01198552 -0.00287498  0.0018547  ... -0.00677586  0.00224516
   0.00565214]
 ...
 [ 0.00318386 -0.0062759  -0.00181069 ... -0.00911282 -0.0040334
  -0.00033792]
 [-0.00141813 -0.00244113 -0.00020297 ... -0.00859972 -0.00341655
  -0.0048064 ]
 [-0.00454862 -0.00553523 -0.00463758 ... -0.00988624 -0.00279619
  -0.00409486]], shape=(100, 65), dtype=float32)


In [19]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 2.3699434e-04  1.5532508e-03 -7.7575087e-03 -1.0771928e-03
 -1.8421619e-03  4.4311495e-03  2.9294649e-03 -2.9211715e-03
  6.6165296e-05  3.2857866e-03  1.7846539e-03  2.3309176e-03
  7.1929446e-03 -2.8733183e-03  1.5713202e-03 -2.2643640e-05
  4.7375401e-03  2.8216743e-03 -9.1942167e-03  4.8925676e-03
 -1.8733254e-03 -6.5270662e-03 -1.3262635e-03  2.7551607e-03
  7.1095824e-03 -9.5927408e-03 -3.3423083e-03  1.8025395e-03
  4.1487915e-03  1.1993058e-03 -8.2307047e-04 -5.5103991e-03
 -4.2274185e-03  4.6198005e-03 -6.7168911e-04 -3.0754639e-03
 -3.3899071e-03 -8.6264178e-04  5.0979150e-03 -4.8956848e-03
  4.8678387e-03 -7.4305204e-03  2.1701052e-03 -3.8972690e-03
  1.0790059e-03 -1.5773443e-03 -5.0332947e-03  3.3746427e-03
  2.2483906e-03  6.9161149e-04 -1.2105416e-03  3.4072492e-03
  2.0619731e-03 -2.3466579e-03  6.5719045e-04 -6.0539721e-03
 -2.6496979e-03  7.6816203e-03 -5.8219419e-04 -2.0552911e-04
  9.4770818e-05  2.4708502e-03  6.2226842e-04  1.0545627e-02
  5.272200

In [20]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"HhiblRbC$tIsDaqs,AOlP'FFS-CbPpp!'L-.&!hOnl$,DuDP;PKh3XIFWmINWpPbgc:UwRXHHsc,tPntUsnRfJZs,$&QPkAMCL&C"

In [21]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [22]:
model.compile(optimizer='adam', loss=loss)

In [23]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [24]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 31s 159ms/step - loss: 2.5540
Epoch 2/50
172/172 [==============================] - 30s 160ms/step - loss: 1.8594
Epoch 3/50
172/172 [==============================] - 30s 161ms/step - loss: 1.6176
Epoch 4/50
172/172 [==============================] - 30s 162ms/step - loss: 1.4883
Epoch 5/50
172/172 [==============================] - 30s 162ms/step - loss: 1.4120
Epoch 6/50
172/172 [==============================] - 30s 162ms/step - loss: 1.3542
Epoch 7/50
172/172 [==============================] - 30s 162ms/step - loss: 1.3096
Epoch 8/50
172/172 [==============================] - 30s 162ms/step - loss: 1.2704
Epoch 9/50
172/172 [==============================] - 30s 162ms/step - loss: 1.2341
Epoch 10/50
172/172 [==============================] - 30s 162ms/step - loss: 1.1968
Epoch 11/50
172/172 [==============================] - 30s 162ms/step - loss: 1.1603
Epoch 12/50
172/172 [==============================] - 30s 162ms/step - lo

In [25]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [26]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
# I think this is if we're dealing with our file

checkpoint_num = 1
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

In [29]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [30]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: romeo
romeous goddess
In every case and join the king.
Come, sir the fair without the which, to deceate,
Say 'pardon' as the uttermors.

KING LEWIS XI:
Whatereon your honour.

LUCIO:
Wife, look you, sirrah! and myself
To wink and brother did not hold on him.

KING LEWIS XI:
How does your little hand?

AUTOLYCUS:
O, good sir, tenderly, as first we were as it is fulfily whilst:
As if he were well; where no hands your house.

LUCENTIO:
I'll geld thee hence.

ANGELO:
Well, I beseech you, let's see when cast matters shall ours.

LADY ANNE:
What is't your honour for the deed.

Second Murderer:
Whom so stoop and enclosses are in arms.

KING RICHARD II:
Down, on the north, e play'd in eate, or both?

KING HENRY VI:
Where did you dwell when?

ESCALUS:
So hast thou there in Bohemia and your father's stand,
As 
